
# Inhaltsverzeichnis

## 4. Korrelationsanalysen

### 4.1 Überblick über Korrelationen
- Berechnung der Pearson-Korrelationsmatrix
- Visualisierung (Heatmap mit unterem Dreieck)
- Dokumentation der stärksten positiven/negativen Korrelationen
- Empfehlung zur Variablenselektion (z. B. PCA als Option)

### 4.2 Visuelle Analyse ausgewählter Variablenpaare
- Pairplots zur Überprüfung auffälliger Korrelationen
- Lineare Regressionslinie + LOWESS (nichtlinearer Trend)
- Manuell ausgewählte Feature-Paare:
  - PM10–PM25
  - NO₂–PM10
  - NO₂–PM25
  - Humidity–PM25
  - O₃–Tavg
  - Tmin–Dew
  - Pres–Dew
  - Pres–Tmin
- Bewertung der Art und Stärke der Zusammenhänge


In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
%matplotlib inline

In [ ]:
# Settings for displaying floats
pd.set_option('display.float_format', '{:,.2f}'.format)

In [ ]:
df = pd.read_csv("./data/cleaned_air_quality_data_2025_03_26.csv")
df.head()

In [ ]:
df.shape

# 4. Korrelationsanalysen

## 4.1 Überblick über die Korrelationen

Die Korrelationsmatrix enthält alle numerischen Features aus dem Datensatz. Als Maßstab für die Stärke einer Korrelation werden folgende Werte angesetzt:

\> 0.8 = strong correlation

0.4 - 0.8 = moderate correlation

< 0.4 = weak correlation

In [ ]:
# calculate correlation matrix (Pearson)
corr_matrix = df.select_dtypes(include=['number']).corr()

# Mask upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Display heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5)
plt.title("Feature Correlations (Pearson)");

In [ ]:
# Show only strong and moderate correlations (>|0.4|); leave out main diagonal (1.0)

# Calculate matrix
corr_matrix = df.select_dtypes(include=['number']).corr()

# extract only strong and moderate correlations (>|0.4|); leave out main diagonal (1.0)
strong_corrs = corr_matrix[(corr_matrix.abs() > 0.4) & (corr_matrix.abs() < 1.0)]

# Convert df to long list (.stack) and reset index
strong_corrs = strong_corrs.stack().reset_index()
strong_corrs.columns = ["Feature 1", "Feature 2", "Korrelation"]

# remove redundant rows (note: the "<"-sign here refers to alphabetic order of feature names, not to numbers of any kind!)
strong_corrs = strong_corrs.loc[strong_corrs["Feature 1"] < strong_corrs["Feature 2"]]

strong_corrs

#### Dokumentation der stärksten positiven/negativen Korrelationen & Empfehlung zur Variablenselektion

Die Korrelationsmatrix zeigt mehrere starke und inhaltlich gut erklärbare Zusammenhänge zwischen den numerischen Variablen. Besonders deutlich sind folgende Muster:

**Feinstaub- und Stickstoffdioxidwerte hängen zusammen:**
Es besteht eine starke Korrelation zwischen PM10 und PM2.5 (r = 0.84), was plausibel ist, da PM2.5 eine Teilmenge von PM10 ist. Zusätzlich korreliert NO₂ moderat mit beiden Feinstaubkomponenten (r ≈ 0.42–0.49), was auf gemeinsame Emissionsquellen wie Verkehr oder Industrie hindeutet.

**Temperaturvariablen sind stark untereinander korreliert:**
Die Tagesmitteltemperatur (Tavg) steht in sehr engem Zusammenhang mit Tmin und Tmax (r ≈ 0.97). Auch Tmin und Tmax selbst sind hoch korreliert (r = 0.90). Das ist mathematisch und physikalisch naheliegend und spricht dafür, nicht alle drei Variablen gleichzeitig zu verwenden, um Redundanz zu vermeiden.

**Der Taupunkt (Dew) korreliert stark mit Temperatur:**
Die stärkste Korrelation liegt zwischen Dew und Tmin (r = 0.87), gefolgt von Tavg (r = 0.82). Dies spiegelt wider, dass die Luftfeuchtigkeit – und damit der Taupunkt – eng mit der Umgebungstemperatur zusammenhängt.

**Einige schwächer negative Korrelationen deuten auf atmosphärische Zusammenhänge hin:**
Der Luftdruck (Pres) korreliert moderat negativ mit Dew (r = –0.43) und Tmin (r = –0.42), was mit typischen meteorologischen Prozessen in Zusammenhang stehen kann (z. B. feuchtwarme Luft in Tiefdruckgebieten).

---

Diese Ergebnisse helfen dabei, hoch korrelierte bzw. redundante Variablen zu erkennen und gezielt für weitere Analysen (z. B. Clusteranalyse oder Modellierung) geeignete Features auszuwählen.

Für viele Verfahren, wie z. B. Clustering oder Regressionsmodelle, ist es ratsam, von stark korrelierten Variablen jeweils nur eine zu verwenden, um Verzerrungen oder sogenannte Multikollinearität zu vermeiden.

Alternativ können Hauptkomponentenanalyse (PCA) oder andere dimensionalitätsreduzierende Verfahren genutzt werden, um mehrere stark korrelierte Variablen zu einer gemeinsamen Komponente zusammenzufassen, ohne wesentliche Information zu verlieren.


### 4.2 Visuelle Analyse ausgewählter Variablenpaare

Zur vertieften Analyse einzelner Zusammenhänge wurden im Folgenden ausgewählte Variablenpaare visuell untersucht. Die Auswahl dieser Paare basiert nicht ausschließlich auf den höchsten Pearson-Korrelationskoeffizienten, sondern folgt einer Kombination aus statistischer Auffälligkeit und inhaltlicher Relevanz. Berücksichtigt wurden sowohl stark korrelierte Variablen (z. B. PM10–PM25), als auch Paare mit moderater Korrelation, deren Beziehung aus umweltwissenschaftlicher Sicht besonders interessant erscheint (z. B. O₃–Tavg oder Humidity–PM25).

Ziel dieser Analyse ist es, nicht nur die Stärke, sondern auch die Form der Zusammenhänge sichtbar zu machen – etwa durch lineare Regressionslinien und LOWESS-Kurven – und gegebenenfalls Hinweise auf nichtlineare oder stadtabhängige Strukturen zu erkennen.

Die Pairplots werden auf einem repräsentativen Sample von 2000 Datenpunkten berechnet.

In [ ]:
features = ["Pm25", "Pm10", "No2", "O3", "Co", "Tavg", "Humidity", "Dew", "Pres"]
sample_df = df[features].dropna().sample(n=2000, random_state=42)
sns.pairplot(sample_df, diag_kind="kde", plot_kws={"alpha": 0.4, "s": 15})

Die Histogramme im Pairplot bestätigen noch einmal die im Notebook 1_eda_exploration berechnete Schiefe vieler Variablen:

Schadstoffe wie Pm25, NO2 und O3 sind erwartungsgemäß rechtsschief verteilt, während Temperatur (Tavg), Luftfeuchtigkeit (Humidity) und Taupunkt (Dew) eine leichte Linksschiefe zeigen. Der Luftdruck (Pres) hebt sich durch seine nahezu normalverteilte Form ab.

#### Pairplots zur Überprüfung auffälliger Korrelationen mit linearer Regressionlinie und/oder LOWESS (nichtlinearer Trend)

Es werden einige Korrelationen herausgegriffen und näher untersucht. Dabei werden nicht Korrelationen in Betracht gezogen, die einen hohen Pearson-Koeffizienten haben (z.B. PM10 vs PM25), sondern auch solche, die möglicherweise keine rein lineare Beziehung haben und dadurch nicht durch einen hohen Pearson-Koeffizienten auffallen.

Um die Art des Zusammenhangs (lienar oder nicht) zwischen zwei Features eindeutig zu identifizieren, werden sowohl lineare Regression als auch LOWESS (Local Weighted Scatterplot Smoothing) angewendet.

Die Samplegröße beträgt jeweils 3000 Datenpunkte. Zur Replizierbarkeit wird ein Random State von 42 festgelegt. NaN-Werte werden für alle Berechnungen entfernt (weil Seaborn damit nicht umgehen kann).

#### 🌫️ PM10 vs PM25 (r = 0.84)

Pm25 ist eine Teilmenge von Pm10. Sind beide Features relevant für spätere Modellierungen oder ist eines davon redundant?

In [ ]:
subset = df[["Pm10", "Pm25"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Pm10", y="Pm25", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "crimson"})

plt.title("LOWESS-Regressionskurve: PM10 vs. PM2.5")
plt.xlabel("PM10-Konzentration (µg/m³)")
plt.ylabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True);

Die LOWESS-Analyse bestätigt den erwarteten sehr starken linearen Zusammenhang zwischen PM10 und PM2.5.

Im zentralen Bereich steigt PM2.5 nahezu proportional zu PM10, mit nur einer leichten Abflachung bei höheren PM10-Werten.
Diese minimale Abweichung könnte darauf hindeuten, dass bei sehr hoher Feinstaubbelastung der relative Anteil von groben Partikeln (PM10 ohne PM2.5) etwas zunimmt.

Aufgrund der wenigen Datenpunkte in den extremen Bereichen kann für diesen keine Belastbare Aussage gemacht werden.

#### 🚗 NO₂ vs PM25 (r = 0,49)

In [ ]:
sample_df = df[["No2", "Pm25"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=sample_df, x="No2", y="Pm25",
            scatter_kws={"alpha":0.3, "s":15},
            line_kws={"color": "darkred"})

plt.title("Zusammenhang zwischen NO₂ und PM2.5 (Stichprobe)")
plt.xlabel("NO₂-Konzentration (µg/m³)")
plt.ylabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True)

Der regplot zwischen NO₂ und PM2.5 zeigt eine positive lineare Tendenz, was plausibel erscheint, da beide Schadstoffe häufig durch Verkehr oder industrielle Prozesse freigesetzt werden.

Aufgrund der breiten Streuung und fehlender Kontextvariablen ist die Regressionslinie jedoch nicht als kausales Modell, sondern lediglich als explorative Visualisierung einer Korrelation zu verstehen.

Der Konfidenzbereich ist nur für den Bereich gut, in dem die meisten Messwerte liegen (Problem: Heteroskadastizität). Die folgende Darstellung zeigt nur diesen zentralen Wertebereich:

In [ ]:
subset = df[(df["No2"] < 40) & (df["Pm25"] < 200)][["No2", "Pm25"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="No2", y="Pm25",
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "darkblue"})

plt.title("Regression innerhalb des zentralen Wertebereichs (No2 < 40, Pm25 < 200)")
plt.xlabel("NO₂-Konzentration (µg/m³)")
plt.ylabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True);

Innerhalb des zentralen Wertebereichs ist eine lineare Verteilung deutlich sichtbar, und auch das Konfidenzintervall ist "akzeptabel".

In [ ]:
subset = df[["No2", "Pm25"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="No2", y="Pm25", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "mediumvioletred"})

plt.title("LOWESS-Regressionskurve für NO₂ und PM2.5")
plt.xlabel("NO₂-Konzentration (µg/m³)")
plt.ylabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True)

Die Anwendung von LOWESS auf den gesamten Wertebereich von NO₂ und PM2.5 zeigt, dass der Zusammenhang nicht durchgängig linear ist.
Besonders im oberen Wertebereich (ab ~70 µg/m³ NO₂) ist die Punktverteilung zu dünn und uneinheitlich, was zu starken Schwankungen in der Regressionskurve führt.

Im zentralen Wertebereich (NO₂ < 40, PM2.5 < 200) zeigt sich hingegen ein stabiler, positiver Trend.

Für lineare Analysen ist daher eine Einschränkung auf diesen Bereich sinnvoll.

#### 🏭 NO₂ vs PM10 (r = 0,42)

Der Zusammenhang sollte ähnlich sein wie der zwischen No2 und PM10.

In [ ]:
subset = df[["No2", "Pm10"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="No2", y="Pm10", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "darkorange"})

plt.title("LOWESS-Regressionskurve für NO₂ und PM10")
plt.xlabel("NO₂-Konzentration (µg/m³)")
plt.ylabel("PM10-Konzentration (µg/m³)")
plt.grid(True);

Die LOWESS-Analyse des Zusammenhangs zwischen NO₂ und PM10 zeigt einen insgesamt positiven Trend, der im Bereich bis etwa 40 µg/m³ NO₂ weitgehend linear verläuft. Ab etwa 40 µg/m³ NO₂ wird der Zusammenhang flacher, was auf eine Sättigung oder zunehmende Streuung hinweisen könnte. In diesem Bereich sind jedoch nur wenige Datenpunkte vorhanden, sodass der Verlauf der Kurve dort nicht belastbar interpretiert werden sollte.

Eine lineare Regression könnte im unteren Bereich sinnvoll sein, sollte aber auf den zentralen Wertebereich beschränkt werden.

--> Der Zusammenhang ist also ähnlich wie der zwischen NO₂ und PM10.

#### 💧 Humidity vs PM25 (r = 0,13)

In [ ]:
subset = df[["Humidity", "Pm25"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Humidity", y="Pm25", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "teal"})

plt.title("LOWESS-Regressionskurve: Luftfeuchtigkeit vs. PM2.5")
plt.xlabel("Relative Luftfeuchtigkeit (%)")
plt.ylabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True);

Die LOWESS-Regression zwischen Luftfeuchtigkeit (Humidity) und PM2.5 zeigt, wie erwartet, einen klar negativen Zusammenhang.
Bei niedriger Luftfeuchtigkeit (unter 20 %) liegen die PM2.5-Werte durchschnittlich bei etwa 70 µg/m³, während sie bei sehr hoher Luftfeuchtigkeit (über 90 %) auf unter 50 µg/m³ sinken.

Dieser Verlauf ist nicht ganz linear, sondern flacht bei zunehmender Feuchtigkeit ab – ein Hinweis auf sättigende Effekte oder natürliche Begrenzungen.

Eine lineare Regression wäre hier möglich, aber nicht unbedingt angemessen – LOWESS zeigt die tatsächliche Struktur des Zusammenhangs besser.

#### ☀️ O₃ vs Tavg

Pearson-Korrelation: 0,19

In [ ]:
subset = df[["O3", "Tavg"]].dropna().sample(n=3000, random_state=42)
plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Tavg", y="O3", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "darkgreen"})

plt.title("Nichtlinearer Zusammenhang zwischen Temperatur (Tavg) und Ozon (O3)")
plt.xlabel("Durchschnittstemperatur (°C)")
plt.ylabel("Ozonkonzentration (µg/m³)")
plt.grid(True);

Die Analyse des Zusammenhangs zwischen Temperatur (Tavg) und Ozonkonzentration (O3) mit Hilfe einer LOWESS-Regression zeigt einen klar nichtlinearen Verlauf:

- Bei Temperaturen zwischen 0 und etwa 20 °C steigt die Ozonkonzentration tendenziell an – ein plausibler Effekt durch sonnengetriebene Bildung.
- Ab ca. 20 °C sinkt die Konzentration jedoch wieder.

Dieser geknickte Verlauf weist darauf hin, dass eine lineare Regressionsanalyse hier nicht geeignet ist, um den Zusammenhang korrekt zu modellieren.

➡️ „Ozon-Peak-Temperatur-Phänomen“

#### 🌡️ Tmin vs Dew (r = 0.87)

Die Temperatur beeinflusst den Taupunkt. Der Taupunkt ist der Punkt, bei dem die Luft gesättigt ist und Wasserdampf kondensiert. Er ist immer kleiner oder gleich der aktuellen Lufttemperatur. Er nähert sich der Lufttemperatur vor allem nachts, wenn es abkühlt. Daher ist der Zusammenhang mit Tmin am stärksten und wird hier exemplarisch ausgewählt.

In [ ]:
subset = df[["Tmin", "Dew"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Tmin", y="Dew", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "mediumseagreen"})

plt.title("LOWESS-Regressionskurve: Tmin vs. Taupunkt (Dew)")
plt.xlabel("Tiefsttemperatur (°C)")
plt.ylabel("Taupunkt (°C)")
plt.grid(True);

Es zeigt sich ein deutlicher linearer Zusammenhang ohne nennenswerte Knicke.

Vergleich von LOWESS und linearer Regressionlinie:

In [ ]:
subset = df[["Tmin", "Dew"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))

# LOWESS
sns.regplot(data=subset, x="Tmin", y="Dew", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "mediumseagreen", "label": "LOWESS"})

# Lineare Regression
sns.regplot(data=subset, x="Tmin", y="Dew", lowess=False,
            scatter=False,
            line_kws={"color": "orangered", "linestyle": "--", "label": "Linear"})

plt.title("Tmin vs. Taupunkt (LOWESS & Lineare Regression)")
plt.xlabel("Tiefsttemperatur (°C)")
plt.ylabel("Taupunkt (°C)")
plt.legend()
plt.grid(True);


Tiefsttemperatur (Tmin) und Taupunkt (Dew) sind hoch korreliert (r = 0.87).

Ein LOWESS-Plot offenbart einen weitgehend linearen Anstieg des Taupunkts mit zunehmender Temperatur, insbesondere im Bereich unter 0 °C. In höheren Temperaturbereichen wird die Steigung etwas flacher, was auf eine schwächer werdende Kopplung hinweisen könnte.

Zum Vergleich wurde zusätzlich eine lineare Regressionslinie mit Konfidenzintervall geplottet. Sie verläuft insgesamt flacher als die LOWESS-Kurve. Das Konfidenzband ist sehr schmal, was auf eine geringe Streuung und hohe Vorhersagbarkeit hinweist.

Der Vergleich zeigt, dass eine lineare Regression im zentralen Bereich zwar möglich, aber nicht ideal ist.
LOWESS eignet sich, um die **leichte** Nichtlinearität sichtbar zu machen und kann für explorative Zwecke empfohlen werden.

#### 📉 Pres vs Dew (r = -0,43)

In [ ]:
subset = df[["Pres", "Dew"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Pres", y="Dew", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "darkorange"})

plt.title("LOWESS-Regressionskurve: Luftdruck vs. Taupunkt")
plt.xlabel("Luftdruck (hPa)")
plt.ylabel("Taupunkt (°C)")
plt.grid(True)

Die LOWESS-Analyse zwischen Luftdruck (Pres) und Taupunkt (Dew) zeigt einen nichtlinearen, umgekehrt U-förmigen Zusammenhang.
Bei niedrigen Druckwerten steigt der Taupunkt zunächst an, flacht im mittleren Bereich ab und fällt schließlich bei hohem Druck steil ab.

Dies unterstützt die meteorologische Annahme, dass Tiefdruck mit feuchterer Luft (höherem Taupunkt) und Hochdruck mit trockenerer Luft (niedrigerem Taupunkt) einhergeht.

#### 🌬️ Pres vs Tmin (r = -0,42)

In [ ]:
subset = df[["Pres", "Tmin"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Pres", y="Tmin", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "darkorange"})

plt.title("LOWESS-Regressionskurve: Luftdruck vs. Tiefsttemperatur")
plt.xlabel("Luftdruck (hPa)")
plt.ylabel("Tiefsttemperatur (°C)")
plt.grid(True)

Die LOWESS-Analyse zwischen Luftdruck (Pres) und Tiefsttemperatur (Tmin) zeigt einen symmetrisch gebogenen Zusammenhang, ähnlich wie bei Pres und Dew.

- In Bereichen mit niedrigem Druck steigen die nächtlichen Tiefsttemperaturen zunächst leicht an, was auf eine isolierende Wirkung von Wolken und Wetteraktivität bei Tiefdruck hindeutet.
- Ab einem Druck von etwa 1008 hPa kehrt sich der Effekt um: Bei hohem Druck sinken die Tiefsttemperaturen, vermutlich durch klare Nächte mit stärkerer Abkühlung.

Die Kurve ist harmonisch geformt und belegt eine meteorologisch plausible, nichtlineare Beziehung.


#### Bewertung der Art und Stärke der Zusammenhänge

Die ausführliche Analyse der Korrelationen – unterstützt durch klassische Korrelationsmatrizen, Regressionsplots und LOWESS-Kurven – hat gezeigt, dass die betrachteten Variablen sinnvolle, teils lineare, teils nichtlineare Zusammenhänge aufweisen, die physikalisch, meteorologisch oder umweltbezogen plausibel erklärbar sind.

Dabei konnten sowohl bekannte Beziehungen (z. B. zwischen Feinstaub und Stickstoffdioxid oder zwischen Taupunkt und Temperatur) als auch weniger offensichtliche, aber nachvollziehbare Muster (z. B. der Einfluss des Luftdrucks auf Temperatur und Feuchtigkeit) sichtbar gemacht werden.

Insgesamt deutet die Korrelationenanalyse darauf hin, dass es sich um natürliche, glaubwürdige und konsistente Umweltdaten handelt, die keine Anzeichen künstlicher Verzerrung oder unplausibler Artefakte zeigen.

Damit ist der Datensatz in seiner bereinigten Form eine solide Grundlage für weiterführende Machine-Learning-Modelle. Die erkannten Strukturen können gezielt genutzt werden, um sinnvolle Features zu definieren, Zusammenhänge zu explorieren und vorhersagende Modelle zu entwickeln, die auf realweltlichen Mustern basieren.